In [6]:
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox

import os
window=Tk()
window.title("Image Analysis")
window.geometry("500x500")




#####Main function for splitting image#####
#####Main function for splitting image#####
#####Main function for splitting image#####
def main(svs_path):
    v.set("Please wait...")
    ###########modules for splitting image##########
    import large_image
    #INFO:girder:Created LRU Cache for 'tilesource' with 167 maximum size
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    %matplotlib inline
    ###########modules for splitting image##########
    
    

    ####Accessing input folder containing all folders having svs files####
    
    main_folder_name = svs_path.split("/")[-1]
    #print("Working on main folder",main_folder_name)
    text.insert(END,"Working on main folder: "+main_folder_name+"\n")
    folder_list = os.listdir(svs_path)

    #splitted_img_path is the main output folder containing all other 
    #folders of each svs file which contain splitted img of respective svs files
    splitted_img_path = svs_path.strip(svs_path.split("/")[-1])+"Splitted_img_"+svs_path.split("/")[-1]
    #print("Output will be generated in: ",splitted_img_path)
    text.insert(END,"Output path: "+splitted_img_path+"\n")
    
    try:
        os.mkdir(splitted_img_path)
    except:
        os.system("rm -r "+ splitted_img_path)
        os.mkdir(splitted_img_path)
        
    
    ####looping over all folders in main input folder####
    ###looking for folders containing svs files ######
    for folder in folder_list:
        
        folder_path = svs_path+"/"+folder
        ####checking whether this path points to folder only or not
        if os.path.isdir(folder_path):
            #print(folder_path)
            files_list = os.listdir(folder_path)
            #print("Working on folder: ",folder)
            
            
        else:
            continue

        ###once confirming the folder path
        ###Accessing the svs file inside this folder

        for file in files_list:
            if file.endswith(".svs"):
                svs_file_path = folder_path + "/" + file
                #splitted_folder_path contains splitted
                #img for a patricular svs file
                splitted_folder_name = svs_file_path.split("/")[-1].strip(".svs")
                splitted_folder_path = splitted_img_path + "/" + splitted_folder_name
                
                os.mkdir(splitted_folder_path)
                

                #loading our image
                try:
                    svs_img = large_image.getTileSource(svs_file_path)
                except:
                    #print("Corrupt image file: ",file)
                    messagebox.showerror("Could not load file", "Corrupt image file: "+file)
                #splitting image into tiles:
                num_tiles_expected = 0
                num_tiles_made=0

                for tile_info in svs_img.tileIterator(
                    region=dict(left=5000, top=5000, width=20000, height=20000, units='base_pixels'),
                    scale=dict(magnification=20),
                    tile_size=dict(width=1000, height=1000),
                    tile_overlap=dict(x=50, y=50),
                    format=large_image.tilesource.TILE_FORMAT_PIL
                ):


                    png_img = np.array(tile_info["tile"])[:,:,:3]
                    #png_file_path is the path of the single png image of a particular svs file
                    png_file_path = splitted_folder_path + "/" + "tile_" + str(num_tiles_made) + ".png"

                    try:
                        plt.imsave(png_file_path,png_img)
                        num_tiles_made+=1
                    except:
                        #print("error while saving file at path: ",png_file_path)
                        messagebox.showerror("Error while saving file at path:",png_file_path)
                        
                        break

                    num_tiles_expected+=1
                    #print("num_tiles_expected: ",num_tiles_expected)
                    #print("num_tiles_made: ",num_tiles_made)
                    #print(out_file_path)
                    break



                #print("no. of tiles={}".format(num_tiles,format = "png"))
                #print(tile_img.shape)
        #print("calling feature_extraction function")
    
    
    #### calling feature extraction function#####
    feature_extraction(splitted_img_path)
#svs_path = "/home/ashish/A"
#main(svs_path)

#####Main function for splitting image end#####
#####Main function for splitting image end#####
#####Main function for splitting image end#####




#####Function for feature extraction#####
#####Function for feature extraction#####
#####Function for feature extraction#####
def feature_extraction(splitted_img_path):
    ###########modules for feature extraction##########
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    %matplotlib inline
    import scipy as sp
    import histomicstk as htk
    import skimage.io
    import skimage.measure
    import skimage.color
    import math
    import statistics
    from scipy.stats import kurtosis, skew, entropy
    ###########modules for feature extraction##########
    
    #give path of the main input folder which contains the respective folder
    #(containing splitted svs image) for all the svs files
    
    
    input_folder_path = splitted_img_path
    
    #storing all the tile_folders inside our main input folder in a list
    tile_folder_list = os.listdir(input_folder_path)
    
    #print(tile_folder_list)

    stats_feature_output_file_path = input_folder_path + "/"+"Statistical_Features_" + ".txt"
    #print(stats_feature_output_file_path)
    
    
    ######Writing headers to the statistical feature output file#####
    fw1 = open(stats_feature_output_file_path,"w")
    fw1.write("file Name\t"+"Average_Major Axis Length\t"+"Average_Minor Axis Length\t"+"Average_ratio of major to minor axis length\t"+"Average_Nuclear Area\t"+"Average_minimum distance to neighboring cells\t"+"Average_maximum distance to neighboring cells\t"+"Average_mean distance to neighboring cells"+"\t"+"Average_red_pixel_trait"+"\t"+"Average_green_pixel_trait"+"\t"+"Average_blue_pixel_trait"+"\t")
    fw1.write("Variance_Major Axis Length\t"+"Variance_Minor Axis Length\t"+"Variance_ratio of major to minor axis length\t"+"Variance_Nuclear Area\t"+"Variance_minimum distance to neighboring cells\t"+"Variance_maximum distance to neighboring cells\t"+"Variance_mean distance to neighboring cells"+"\t"+"Variance_red_pixel_trait"+"\t"+"Variance_green_pixel_trait"+"\t"+"Variance_blue_pixel_trait"+"\t")
    fw1.write("Stdev_Major Axis Length\t"+"Stdev_Minor Axis Length\t"+"Stdev_ratio of major to minor axis length\t"+"Stdev_Nuclear Area\t"+"Stdev_minimum distance to neighboring cells\t"+"Stdev_maximum distance to neighboring cells\t"+"Stdev_mean distance to neighboring cells"+"\t"+"Stdev_red_pixel_trait"+"\t"+"Stdev_green_pixel_trait"+"\t"+"Stdev_blue_pixel_trait"+"\t")
    fw1.write("Skewness_Major Axis Length\t"+"Skewness_Minor Axis Length\t"+"Skewness_ratio of major to minor axis length\t"+"Skewness_Nuclear Area\t"+"Skewness_minimum distance to neighboring cells\t"+"Skewness_maximum distance to neighboring cells\t"+"Skewness_mean distance to neighboring cells"+"\t"+"Skewness_red_pixel_trait"+"\t"+"Skewness_green_pixel_trait"+"\t"+"Skewness_blue_pixel_trait"+"\t")
    fw1.write("Kurtosis_Major Axis Length\t"+"Kurtosis_Minor Axis Length\t"+"Kurtosis_ratio of major to minor axis length\t"+"Kurtosis_Nuclear Area\t"+"Kurtosis_minimum distance to neighboring cells\t"+"Kurtosis_maximum distance to neighboring cells\t"+"Kurtosis_mean distance to neighboring cells"+"\t"+"Kurtosis_red_pixel_trait"+"\t"+"Kurtosis_green_pixel_trait"+"\t"+"Kurtosis_blue_pixel_trait"+"\t")
    fw1.write("Entropy_Major Axis Length\t"+"Entropy_Minor Axis Length\t"+"Entropy_ratio of major to minor axis length\t"+"Entropy_Nuclear Area\t"+"Entropy_minimum distance to neighboring cells\t"+"Entropy_maximum distance to neighboring cells\t"+"Entropy_mean distance to neighboring cells"+"\t"+"Entropy_red_pixel_trait"+"\t"+"Entropy_green_pixel_trait"+"\t"+"Entropy_blue_pixel_trait"+"\n")



    #####loading reference image#####
    #####this image is used in preprocessing of our input image
    ref_image_path = ('https://data.kitware.com/api/v1/file/'
                              '57718cc28d777f1ecd8a883c/download')


    #looping over all the tile_folders inside our main input folder
    #each tile_folder contains all the png files of a particular svs file
    for tile_folder in tile_folder_list:
        print(tile_folder)

        tile_folder_path = input_folder_path + "/" + tile_folder
        #print(tile_folder_path)

        if not(os.path.isdir(tile_folder_path)):
            continue
        
    
        
        nu_feature_output_file_path = input_folder_path + "/"+"Nuclear_Features_" + tile_folder + ".txt"
        #print(nu_feature_output_file_path)



        #Storing all the png files in a particular tile_folder in a list
        png_file_list = os.listdir(tile_folder_path)
        #print(png_file_list)

        ######Writing headers to the nuclear feature output file#####
        
        fw = open(nu_feature_output_file_path,"w")

        fw.write("Major_Axis"+"\t"+"Minor_Axis"+"\t"+"Ratio"+"\t"+"Area"+"\t"+"Mean_Distance"+"\t"+"Max_Distance"+"\t"+"Min_Distance"+"\t"+"Red_Color_Trait"+"\t"+"Green_Color_Trait"+"\t"+"Blue_Color_Trait"+"\n")



        #the following list will contain data for all the png files of a particular svs
        maj_axis_list=[]
        min_axis_list=[]
        ratio_list=[]
        nu_area_list=[]
        min_dist_list=[]
        max_dist_list=[]
        mean_dist_list=[]
        red_pixel_trait_list=[]
        green_pixel_trait_list=[]
        blue_pixel_trait_list=[]

        #Accessing each png file from within the tile folders:
        for png_file in png_file_list:
            #loading inputimage:
            png_file_path = tile_folder_path+"/"+png_file
            print(png_file_path)
            input_img = skimage.io.imread(png_file_path)[:, :, :3]
            #plt.imshow(input_img)


            #####Pre-Processing#####

            titlesize = 24


            ref_img = skimage.io.imread(ref_image_path)[:, :, :3]
            # get mean and stddev of reference image in lab space
            mean_ref, std_ref = htk.preprocessing.color_conversion.lab_mean_std(ref_img)

            # perform reinhard color normalization
            im_nmzd = htk.preprocessing.color_normalization.reinhard(input_img, mean_ref, std_ref)

            # Display results
            #plt.figure(figsize=(20, 10))

            #plt.subplot(1, 2, 1)
            #plt.imshow(ref_img)
            #_ = plt.title('Reference Image', fontsize=titlesize)

            #plt.subplot(1, 2, 2)
            #plt.imshow(im_nmzd)
            #_ = plt.title('Normalized Input Image', fontsize=titlesize)




            #color deconvolution:

            # create stain to color map
            stainColorMap = {
                'hematoxylin': [0.65, 0.70, 0.29],
                'eosin':       [0.07, 0.99, 0.11],
                'dab':         [0.27, 0.57, 0.78],
                'null':        [0.0, 0.0, 0.0]
            }

            # specify stains of input image
            stain_1 = 'hematoxylin'   # nuclei stain
            stain_2 = 'eosin'         # cytoplasm stain
            stain_3 = 'null'          # set to null of input contains only two stains

            # create stain matrix
            W = np.array([stainColorMap[stain_1],
                          stainColorMap[stain_2],
                          stainColorMap[stain_3]]).T

            # perform standard color deconvolution
            im_stains = htk.preprocessing.color_deconvolution.color_deconvolution(im_nmzd, W).Stains

            # Display results
            #plt.figure(figsize=(20, 10))

            #plt.subplot(1, 2, 1)
            #plt.imshow(im_stains[:, :, 0])
            #plt.title(stain_1, fontsize=titlesize)

            #plt.subplot(1, 2, 2)
            #plt.imshow(im_stains[:, :, 1])
            #_ = plt.title(stain_2, fontsize=titlesize)





            #nuclei segmentation:
            # get nuclei/hematoxylin channel
            im_nuclei_stain = im_stains[:, :, 0]

            # segment foreground
            foreground_threshold = 60

            im_fgnd_mask = sp.ndimage.morphology.binary_fill_holes(
                im_nuclei_stain < foreground_threshold)

            # run adaptive multi-scale LoG filter
            min_radius = 10
            max_radius = 15

            im_log_max, im_sigma_max = htk.filters.shape.cdog(
                im_nuclei_stain, im_fgnd_mask,
                sigma_min=min_radius * np.sqrt(2),
                sigma_max=max_radius * np.sqrt(2)
            )

            # detect and segment nuclei using local maximum clustering
            local_max_search_radius = 10

            im_nuclei_seg_mask, seeds, maxima = htk.segmentation.nuclear.max_clustering(
               im_log_max, im_fgnd_mask, local_max_search_radius)

            # filter out small objects
            min_nucleus_area = 80

            im_nuclei_seg_mask = htk.segmentation.label.area_open(
                im_nuclei_seg_mask, min_nucleus_area).astype(np.int)

            # compute nuclei properties
            objProps = skimage.measure.regionprops(im_nuclei_seg_mask)

            print('Number of nuclei =',len(objProps))
            # Display results
            #plt.figure(figsize=(20, 10))

            #plt.subplot(1, 2, 1)
            #plt.imshow(skimage.color.label2rgb(im_nuclei_seg_mask, input_img, bg_label=0), origin='lower')
            #plt.title('Nuclei segmentation mask overlay', fontsize=titlesize)

            #plt.subplot(1, 2, 2)
            #plt.imshow( input_img )
            #plt.xlim([0, input_img.shape[1]])
            #plt.ylim([0, input_img.shape[0]])
            #plt.title('Nuclei bounding boxes', fontsize=titlesize)

            for i in range(len(objProps)):

                c = [objProps[i].centroid[1], objProps[i].centroid[0], 0]
                width = objProps[i].bbox[3] - objProps[i].bbox[1] + 1
                height = objProps[i].bbox[2] - objProps[i].bbox[0] + 1

                cur_bbox = {
                    "type":        "rectangle",
                    "center":      c,
                    "width":       width,
                    "height":      height,
                }

                #plt.plot(c[0], c[1], 'g+')
                mrect = mpatches.Rectangle([c[0] - 0.5 * width, c[1] - 0.5 * height] ,
                                           width, height, fill=False, ec='g', linewidth=2)
                #plt.gca().add_patch(mrect)







                ########Feature Extraction ########

            #######Extracting major, minor axis, ratio of major to minor axis, and area for each nuclei


            for i in range(len(objProps)):
                major_axis = objProps[i].major_axis_length
                maj_axis_list.append(major_axis)
                minor_axis = objProps[i].minor_axis_length
                min_axis_list.append(minor_axis)
                ratio_maj_min = major_axis/minor_axis
                ratio_list.append(ratio_maj_min)
                nuclear_area = objProps[i].area
                nu_area_list.append(nuclear_area)



            #### finding min dist and max of each nuclei#####

                x,y = objProps[i].centroid
                dist_list = [] #contains dist of a single nucle from each nuclei

                for j in range(len(objProps)):
                    if i != j :
                        x1,y1 = objProps[j].centroid
                        dist = math.sqrt((y1-y)**2+(x1-x)**2)
                        dist_list.append(dist)

                min_dist = min(dist_list) #gives min dist for a particular nuclei with all other nuclei
                min_dist_list.append(min_dist)
                max_dist = max(dist_list)#gives max dist for a particular nuclei with all other nuclei
                max_dist_list.append(max_dist)
                mean_dist = statistics.mean(dist_list)#gives mean dist for a particular nuclei with all other nuclei
                mean_dist_list.append(mean_dist)


            ####finding color traits of each nuclei 
                x,y = objProps[i].centroid
                r,g,b = input_img[int(x),int(y)]
                red_pixel_trait_list.append(int(r))
                green_pixel_trait_list.append(int(g))
                blue_pixel_trait_list.append(int(b))


            ###### Writing all the data to nuclear feature file ########

                fw.write(str(major_axis)+"\t"+str(minor_axis)+"\t"+str(ratio_maj_min)+"\t"+str(nuclear_area)+"\t"+str(mean_dist)+"\t"+str(max_dist)+"\t"+str(min_dist)+"\t"+str(r)+"\t"+str(g)+"\t"+str(b)+"\n")

        fw.close()
        
        #####calculating statistical data from combined nuclear data of all the png of a particular svs

        ###average:
        maj_axis_avg=statistics.mean(maj_axis_list)
        min_axis_avg=statistics.mean(min_axis_list)
        ratio_avg=statistics.mean(ratio_list)
        nu_area_avg=statistics.mean(nu_area_list)
        min_dist_avg=statistics.mean(min_dist_list)
        max_dist_avg=statistics.mean(max_dist_list)
        mean_dist_avg=statistics.mean(mean_dist_list)
        R_pixel_trait_avg=statistics.mean(red_pixel_trait_list)
        G_pixel_trait_avg=statistics.mean(green_pixel_trait_list)
        B_pixel_trait_avg=statistics.mean(blue_pixel_trait_list)

        ###variance:
        maj_axis_variance=statistics.variance(maj_axis_list)
        min_axis_variance=statistics.variance(min_axis_list)
        ratio_variance=statistics.variance(ratio_list)
        nu_area_variance=statistics.variance(nu_area_list)
        min_dist_variance=statistics.variance(min_dist_list)
        max_dist_variance=statistics.variance(max_dist_list)
        mean_dist_variance=statistics.variance(mean_dist_list)
        R_pixel_trait_variance=statistics.variance(red_pixel_trait_list)
        G_pixel_trait_variance=statistics.variance(green_pixel_trait_list)
        B_pixel_trait_variance=statistics.variance(blue_pixel_trait_list)

        ###stdev:
        maj_axis_stdev=statistics.stdev(maj_axis_list)
        min_axis_stdev=statistics.stdev(min_axis_list)
        ratio_stdev=statistics.stdev(ratio_list)
        nu_area_stdev=statistics.stdev(nu_area_list)
        min_dist_stdev=statistics.stdev(min_dist_list)
        max_dist_stdev=statistics.stdev(max_dist_list)
        mean_dist_stdev=statistics.stdev(mean_dist_list)
        R_pixel_trait_stdev=statistics.stdev(red_pixel_trait_list)
        G_pixel_trait_stdev=statistics.stdev(green_pixel_trait_list)
        B_pixel_trait_stdev=statistics.stdev(blue_pixel_trait_list)


        ###skewness:
        maj_axis_skew=skew(maj_axis_list)
        min_axis_skew=skew(min_axis_list)
        ratio_skew=skew(ratio_list)
        nu_area_skew=skew(nu_area_list)
        min_dist_skew=skew(min_dist_list)
        max_dist_skew=skew(max_dist_list)
        mean_dist_skew=skew(mean_dist_list)
        R_pixel_trait_skew=skew(red_pixel_trait_list)
        G_pixel_trait_skew=skew(green_pixel_trait_list)
        B_pixel_trait_skew=skew(blue_pixel_trait_list)

        ###Kurtosis:
        maj_axis_kurtosis=kurtosis(maj_axis_list)
        min_axis_kurtosis=kurtosis(min_axis_list)
        ratio_kurtosis=kurtosis(ratio_list)
        nu_area_kurtosis=kurtosis(nu_area_list)
        min_dist_kurtosis=kurtosis(min_dist_list)
        max_dist_kurtosis=kurtosis(max_dist_list)
        mean_dist_kurtosis=kurtosis(mean_dist_list)
        R_pixel_trait_kurtosis=kurtosis(red_pixel_trait_list)
        G_pixel_trait_kurtosis=kurtosis(green_pixel_trait_list)
        B_pixel_trait_kurtosis=kurtosis(blue_pixel_trait_list)

        ###Entropy:
        maj_axis_entropy=entropy(maj_axis_list)
        min_axis_entropy=entropy(min_axis_list)
        ratio_entropy=entropy(ratio_list)
        nu_area_entropy=entropy(nu_area_list)
        min_dist_entropy=entropy(min_dist_list)
        max_dist_entropy=entropy(max_dist_list)
        mean_dist_entropy=entropy(mean_dist_list)
        R_pixel_trait_entropy=entropy(red_pixel_trait_list)
        G_pixel_trait_entropy=entropy(green_pixel_trait_list)
        B_pixel_trait_entropy=entropy(blue_pixel_trait_list)





        ####Writing file name to statistical feature file####
        fw1.write(tile_folder+".svs"+"\t")



        ###### Writing all the data to statistical feature file ########

        ####Writing average:
        fw1.write(str(maj_axis_avg)+"\t")
        fw1.write(str(min_axis_avg)+"\t")
        fw1.write(str(ratio_avg)+"\t")
        fw1.write(str(nu_area_avg)+"\t")
        fw1.write(str(min_dist_avg)+"\t")
        fw1.write(str(max_dist_avg)+"\t")
        fw1.write(str(mean_dist_avg)+"\t")
        fw1.write(str(R_pixel_trait_avg)+"\t")
        fw1.write(str(G_pixel_trait_avg)+"\t")
        fw1.write(str(B_pixel_trait_avg)+"\t")

        ####Writing variance:
        fw1.write(str(maj_axis_variance)+"\t")
        fw1.write(str(min_axis_variance)+"\t")
        fw1.write(str(ratio_variance)+"\t")
        fw1.write(str(nu_area_variance)+"\t")
        fw1.write(str(min_dist_variance)+"\t")
        fw1.write(str(max_dist_variance)+"\t")
        fw1.write(str(mean_dist_variance)+"\t")
        fw1.write(str(R_pixel_trait_variance)+"\t")
        fw1.write(str(G_pixel_trait_variance)+"\t")
        fw1.write(str(B_pixel_trait_variance)+"\t")

        ####Writing stdev:
        fw1.write(str(maj_axis_stdev)+"\t")
        fw1.write(str(min_axis_stdev)+"\t")
        fw1.write(str(ratio_stdev)+"\t")
        fw1.write(str(nu_area_stdev)+"\t")
        fw1.write(str(min_dist_stdev)+"\t")
        fw1.write(str(max_dist_stdev)+"\t")
        fw1.write(str(mean_dist_stdev)+"\t")
        fw1.write(str(R_pixel_trait_stdev)+"\t")
        fw1.write(str(G_pixel_trait_stdev)+"\t")
        fw1.write(str(B_pixel_trait_stdev)+"\t")

        ####Writing skewness:
        fw1.write(str(maj_axis_skew)+"\t")
        fw1.write(str(min_axis_skew)+"\t")
        fw1.write(str(ratio_skew)+"\t")
        fw1.write(str(nu_area_skew)+"\t")
        fw1.write(str(min_dist_skew)+"\t")
        fw1.write(str(max_dist_skew)+"\t")
        fw1.write(str(mean_dist_skew)+"\t")
        fw1.write(str(R_pixel_trait_skew)+"\t")
        fw1.write(str(G_pixel_trait_skew)+"\t")
        fw1.write(str(B_pixel_trait_skew)+"\t")

        ####Writing kurtosis:
        fw1.write(str(maj_axis_kurtosis)+"\t")
        fw1.write(str(min_axis_kurtosis)+"\t")
        fw1.write(str(ratio_kurtosis)+"\t")
        fw1.write(str(nu_area_kurtosis)+"\t")
        fw1.write(str(min_dist_kurtosis)+"\t")
        fw1.write(str(max_dist_kurtosis)+"\t")
        fw1.write(str(mean_dist_kurtosis)+"\t")
        fw1.write(str(R_pixel_trait_kurtosis)+"\t")
        fw1.write(str(G_pixel_trait_kurtosis)+"\t")
        fw1.write(str(B_pixel_trait_kurtosis)+"\t")

        ####Writing entropy:
        fw1.write(str(maj_axis_entropy)+"\t")
        fw1.write(str(min_axis_entropy)+"\t")
        fw1.write(str(ratio_entropy)+"\t")
        fw1.write(str(nu_area_entropy)+"\t")
        fw1.write(str(min_dist_entropy)+"\t")
        fw1.write(str(max_dist_entropy)+"\t")
        fw1.write(str(mean_dist_entropy)+"\t")
        fw1.write(str(R_pixel_trait_entropy)+"\t")
        fw1.write(str(G_pixel_trait_entropy)+"\t")
        fw1.write(str(B_pixel_trait_entropy)+"\t")


        fw1.write("\n")
    fw1.close()
    v.set("Process completed")
    messagebox.showinfo("Process completed","Output text files path: "+input_folder_path)


#####Function for feature extraction end#####
#####Function for feature extraction end#####
#####Function for feature extraction end#####    
    



def browse_directory():

    directory = filedialog.askdirectory(initialdir="/home",title='Please select a directory')
    #print(directory)
    entry1.delete(0,END)
    entry1.insert(END,directory)
    
def main_caller():
    svs_path = entry1.get()
    #print(svs_path)
    main(svs_path)
    
    
######image_frame_start######
image_frame=Frame(window,borderwidth=5,relief="sunken",width=50, height=50)
image_frame.pack(side="top",fill="x")
i=PhotoImage(file="/home/ashish/image_analysis/gui_img.png")
image_label=Label(image_frame,image = i)
image_label.pack(anchor="w",fill="both")
######image_frame_ends###### 

######frame1_start######
frame1=Frame(window,relief="sunken",borderwidth=5)
frame1.pack(side="top",pady=2)

label1=Label(frame1,text="Browse Input Folder")
label1.grid(row=0,column=0,padx=5,pady=5)
button1=Button(frame1,text="Browse",bg="black",fg="white",command=browse_directory)
button1.grid(row=0,column=2)
entry1 = Entry(frame1)
entry1.grid(row=0,column=1,padx=5,pady=5)

button2=Button(frame1,text="Generate Output",bg="black",fg="white",command=main_caller)
button2.grid(row=1,column=1,padx=5,pady=5)
v = StringVar()
v.set("Status")

label2=Label(frame1,textvariable=v)
label2.grid(row=2,column=1,padx=5,pady=5)
######frame1_end######


######frame2_start######
frame2=Frame(window,relief="sunken",borderwidth=5)
frame2.pack(side="top",pady=2,fill="x")
text = Text(frame2)
text.pack(side="top",fill="x")
######frame2_end######







window.mainloop()

TCGA-03-0010-01Z-00-DX4.07de2e55-a8fe-40ee-9e98-bcb78050b9f7
/home/ashish/Splitted_img_A/TCGA-03-0010-01Z-00-DX4.07de2e55-a8fe-40ee-9e98-bcb78050b9f7/tile_0.png
Number of nuclei = 174
TCGA-02-0010-01Z-00-DX4.07de2e55-a8fe-40ee-9e98-bcb78050b9f7
/home/ashish/Splitted_img_A/TCGA-02-0010-01Z-00-DX4.07de2e55-a8fe-40ee-9e98-bcb78050b9f7/tile_0.png
Number of nuclei = 174
